<a href="https://colab.research.google.com/github/its-Shivam732/XrayDenoise/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install torch torchvision diffusers accelerate scikit-image pytorch-fid pillow -q

# Import libraries
import os
import torch
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage.util import random_noise
from google.colab import drive
import glob
from PIL import Image
import shutil
import zipfile

# Mount Google Drive for persistent storage
drive.mount('/content/drive', force_remount=True)
output_dir = '/content/drive/MyDrive/xray_denoising_dataset'
os.makedirs(output_dir, exist_ok=True)

# ============================================
# DOWNLOAD DATASET USING KAGGLE API
# ============================================

# Setup Kaggle API
os.makedirs('/root/.kaggle', exist_ok=True)

# Upload your kaggle.json file
print("Please upload your kaggle.json file:")
print("1. Go to https://www.kaggle.com/settings/account")
print("2. Scroll to 'API' section and click 'Create New Token'")
print("3. Upload the downloaded kaggle.json file when prompted below")

from google.colab import files
uploaded = files.upload()  # This will prompt you to upload kaggle.json

# Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
dataset_root = '/content/covid_xray/COVID-19_Radiography_Dataset'

if not os.path.exists(dataset_root):
    print("\nDownloading COVID-19 Radiography Dataset from Kaggle...")

    # Download dataset
    !kaggle datasets download -d tawsifurrahman/covid19-radiography-database -p /content/covid_xray

    # Extract the zip file
    print("\nExtracting dataset...")
    zip_path = '/content/covid_xray/covid19-radiography-database.zip'

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/covid_xray')

    # Remove zip file to save space
    os.remove(zip_path)

    print("✓ Dataset downloaded and extracted successfully!")

    # Verify the structure
    if os.path.exists(dataset_root):
        print(f"✓ Dataset verified at {dataset_root}")
    else:
        # Sometimes the extracted folder has a different name
        # List what's actually in the directory
        print("\nContents of /content/covid_xray:")
        !ls -la /content/covid_xray

        # Try to find the actual dataset folder
        for item in os.listdir('/content/covid_xray'):
            item_path = os.path.join('/content/covid_xray', item)
            if os.path.isdir(item_path):
                print(f"\nFound directory: {item_path}")
                # Update dataset_root to the actual path
                dataset_root = item_path
                break
else:
    print(f"✓ Dataset already exists at {dataset_root}")

# Verify dataset directory
if not os.path.exists(dataset_root):
    raise FileNotFoundError(f"Dataset root not found at {dataset_root}")

# Get all image paths from subdirectories
def collect_image_paths(root_dir):
    """Collect all image paths from the dataset structure"""
    image_paths = []
    categories = ['Normal', 'COVID', 'Lung_Opacity', 'Viral Pneumonia']

    for category in categories:
        images_dir = os.path.join(root_dir, category, 'images')
        if os.path.exists(images_dir):
            # Get all PNG and JPEG files
            png_files = glob.glob(os.path.join(images_dir, '*.png'))
            jpg_files = glob.glob(os.path.join(images_dir, '*.jpg'))
            jpeg_files = glob.glob(os.path.join(images_dir, '*.jpeg'))

            category_images = png_files + jpg_files + jpeg_files
            image_paths.extend(category_images)
            print(f"Found {len(category_images)} images in {category}")

    return image_paths

# Continue with rest of your code...

Mounted at /content/drive
Please upload your kaggle.json file:
1. Go to https://www.kaggle.com/settings/account
2. Scroll to 'API' section and click 'Create New Token'
3. Upload the downloaded kaggle.json file when prompted below


Saving kaggle.json to kaggle.json

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
License(s): copyright-authors
 94% 728M/778M [00:07<00:00, 223MB/s]
100% 778M/778M [00:07<00:00, 107MB/s]

Extracting dataset...
✓ Dataset downloaded and extracted successfully!
✓ Dataset verified at /content/covid_xray/COVID-19_Radiography_Dataset


In [2]:
# Install dependencies
!pip install torch torchvision diffusers accelerate scikit-image pytorch-fid pillow

# Import libraries
import os
import torch
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage.util import random_noise
from google.colab import drive
import glob
from PIL import Image

# Mount Google Drive for persistent storage
drive.mount('/content/drive', force_remount=True)
output_dir = '/content/drive/MyDrive/xray_denoising_dataset'
os.makedirs(output_dir, exist_ok=True)

# Verify dataset directory
dataset_root = '/content/covid_xray/COVID-19_Radiography_Dataset'
if not os.path.exists(dataset_root):
    raise FileNotFoundError(f"Dataset root not found at {dataset_root}")

# Get all image paths from subdirectories
def collect_image_paths(root_dir):
    """Collect all image paths from the dataset structure"""
    image_paths = []
    categories = ['Normal', 'COVID', 'Lung_Opacity', 'Viral_Pneumonia']

    for category in categories:
        images_dir = os.path.join(root_dir, category, 'images')
        if os.path.exists(images_dir):
            # Get all PNG and JPEG files
            png_files = glob.glob(os.path.join(images_dir, '*.png'))
            jpg_files = glob.glob(os.path.join(images_dir, '*.jpg'))
            jpeg_files = glob.glob(os.path.join(images_dir, '*.jpeg'))

            category_images = png_files + jpg_files + jpeg_files
            image_paths.extend(category_images)
            print(f"Found {len(category_images)} images in {category}")

    return image_paths

# Collect all image paths
all_image_paths = collect_image_paths(dataset_root)
print(f"\nTotal images found: {len(all_image_paths)}")

if len(all_image_paths) == 0:
    raise FileNotFoundError("No images found! Check the directory structure.")

# Limit to 2000 images for faster processing
all_image_paths = all_image_paths[:2000]
print(f"Using {len(all_image_paths)} images for this run")

# Custom Dataset class
class XRayDataset(Dataset):
    def __init__(self, image_paths, transform=None, noise_type='gaussian', noise_level=0.1):
        self.image_paths = image_paths
        self.transform = transform
        self.noise_type = noise_type
        self.noise_level = noise_level

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            # Load image
            image_path = self.image_paths[idx]
            image = Image.open(image_path).convert('L')  # Grayscale

            # Resize to 256x256
            image = image.resize((256, 256), Image.Resampling.LANCZOS)

            # Convert to numpy array and normalize to [0,1]
            image = np.array(image, dtype=np.float32) / 255.0

            # Add noise
            if self.noise_type == 'gaussian':
                noisy = random_noise(image, mode='gaussian', var=self.noise_level**2, clip=True)
            elif self.noise_type == 'poisson':
                noisy = random_noise(image, mode='poisson', clip=True)
            else:
                noisy = image.copy()

            # Convert to tensors [1, H, W]
            image_tensor = torch.from_numpy(image).unsqueeze(0).float()
            noisy_tensor = torch.from_numpy(noisy.astype(np.float32)).unsqueeze(0).float()

            # Apply normalization to [-1, 1] if transform provided
            if self.transform:
                image_tensor = self.transform(image_tensor)
                noisy_tensor = self.transform(noisy_tensor)

            return {'clean': image_tensor, 'noisy': noisy_tensor}

        except Exception as e:
            print(f"Error processing image {idx} ({self.image_paths[idx]}): {e}")
            # Return a blank image pair as fallback
            blank = torch.zeros(1, 256, 256)
            return {'clean': blank, 'noisy': blank}

# Preprocessing transform (normalization only)
transform = transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]

# Split dataset
train_size = 1500
val_size = min(500, len(all_image_paths) - train_size)

train_paths = all_image_paths[:train_size]
val_paths = all_image_paths[train_size:train_size + val_size]

print(f"\nCreating datasets...")
print(f"Train: {len(train_paths)} images")
print(f"Val: {len(val_paths)} images")

# Create datasets
train_dataset = XRayDataset(train_paths, transform=transform, noise_type='gaussian', noise_level=0.1)
val_dataset = XRayDataset(val_paths, transform=transform, noise_type='gaussian', noise_level=0.1)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

print(f"\nDataLoaders created successfully!")

# Save sample images
def save_samples(loader, num_samples=5, output_dir=output_dir):
    samples_dir = os.path.join(output_dir, 'samples')
    os.makedirs(samples_dir, exist_ok=True)

    try:
        batch = next(iter(loader))
        num_to_save = min(num_samples, len(batch['clean']))

        for i in range(num_to_save):
            # Denormalize from [-1, 1] to [0, 1]
            clean = (batch['clean'][i].squeeze().numpy() * 0.5 + 0.5).clip(0, 1)
            noisy = (batch['noisy'][i].squeeze().numpy() * 0.5 + 0.5).clip(0, 1)

            # Create comparison figure
            plt.figure(figsize=(10, 5))

            plt.subplot(1, 2, 1)
            plt.imshow(clean, cmap='gray', vmin=0, vmax=1)
            plt.title('Clean X-Ray', fontsize=14)
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(noisy, cmap='gray', vmin=0, vmax=1)
            plt.title(f'Noisy (Gaussian σ=0.1)', fontsize=14)
            plt.axis('off')

            plt.tight_layout()
            plt.savefig(os.path.join(samples_dir, f'sample_{i}.png'), dpi=150, bbox_inches='tight')
            plt.close()

        print(f"Saved {num_to_save} sample pairs to {samples_dir}")

    except Exception as e:
        print(f"Error saving samples: {e}")

# Save samples from training set
print("\nGenerating sample images...")
save_samples(train_loader, num_samples=8)

# Save metadata
metadata_path = os.path.join(output_dir, 'metadata.txt')
with open(metadata_path, 'w') as f:
    f.write('X-Ray Denoising Dataset\n')
    f.write('='*50 + '\n\n')
    f.write(f'Source: COVID-19 Radiography Database\n')
    f.write(f'Dataset root: {dataset_root}\n\n')
    f.write(f'Training images: {len(train_paths)}\n')
    f.write(f'Validation images: {len(val_paths)}\n')
    f.write(f'Total images: {len(all_image_paths)}\n\n')
    f.write(f'Image specifications:\n')
    f.write(f'  - Size: 256x256 pixels\n')
    f.write(f'  - Format: Grayscale\n')
    f.write(f'  - Normalization: [-1, 1]\n\n')
    f.write(f'Noise configuration:\n')
    f.write(f'  - Type: Gaussian\n')
    f.write(f'  - Standard deviation (σ): 0.1\n')
    f.write(f'  - Variance: 0.01\n\n')
    f.write(f'DataLoader settings:\n')
    f.write(f'  - Batch size: 16\n')
    f.write(f'  - Num workers: 2\n')
    f.write(f'  - Pin memory: True\n')

print(f"\nMetadata saved to {metadata_path}")

# Test loading a batch
print("\nTesting data loading...")
try:
    test_batch = next(iter(train_loader))
    print(f"✓ Successfully loaded batch!")
    print(f"  - Clean images shape: {test_batch['clean'].shape}")
    print(f"  - Noisy images shape: {test_batch['noisy'].shape}")
    print(f"  - Clean images range: [{test_batch['clean'].min():.3f}, {test_batch['clean'].max():.3f}]")
    print(f"  - Noisy images range: [{test_batch['noisy'].min():.3f}, {test_batch['noisy'].max():.3f}]")
except Exception as e:
    print(f"✗ Error loading batch: {e}")

# Initialize PyTorch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n{'='*50}")
print(f"Setup complete!")
print(f"Device: {device}")
print(f"Output directory: {output_dir}")
print(f"{'='*50}\n")

# Print summary
print("Dataset is ready for training!")
print(f"Access training data with: train_loader")
print(f"Access validation data with: val_loader")
print(f"Check samples at: {os.path.join(output_dir, 'samples')}")

Mounted at /content/drive
Found 10192 images in Normal
Found 3616 images in COVID
Found 6012 images in Lung_Opacity

Total images found: 19820
Using 2000 images for this run

Creating datasets...
Train: 1500 images
Val: 500 images

DataLoaders created successfully!

Generating sample images...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved 8 sample pairs to /content/drive/MyDrive/xray_denoising_dataset/samples

Metadata saved to /content/drive/MyDrive/xray_denoising_dataset/metadata.txt

Testing data loading...
✓ Successfully loaded batch!
  - Clean images shape: torch.Size([16, 1, 256, 256])
  - Noisy images shape: torch.Size([16, 1, 256, 256])
  - Clean images range: [-1.000, 1.000]
  - Noisy images range: [-1.000, 1.000]

Setup complete!
Device: cpu
Output directory: /content/drive/MyDrive/xray_denoising_dataset

Dataset is ready for training!
Access training data with: train_loader
Access validation data with: val_loader
Check samples at: /content/drive/MyDrive/xray_denoising_dataset/samples


In [3]:
# Install additional packages if needed
!pip install scikit-image -q

# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

print("✓ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


✓ All libraries imported successfully!
PyTorch version: 2.8.0+cu126
CUDA available: False


In [4]:
# ============================================================================
# CELL 1: Data Preparation (Run after dataset download)
# ============================================================================

# Collect all image paths
all_image_paths = collect_image_paths(dataset_root)
print(f"\nTotal images found: {len(all_image_paths)}")

if len(all_image_paths) == 0:
    raise FileNotFoundError("No images found! Check the directory structure.")

# Limit to 2000 images for faster processing
all_image_paths = all_image_paths[:2000]
print(f"Using {len(all_image_paths)} images for this run")

# Custom Dataset class
class XRayDataset(Dataset):
    def __init__(self, image_paths, transform=None, noise_type='gaussian', noise_level=0.1):
        self.image_paths = image_paths
        self.transform = transform
        self.noise_type = noise_type
        self.noise_level = noise_level

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            # Load image
            image_path = self.image_paths[idx]
            image = Image.open(image_path).convert('L')  # Grayscale

            # Resize to 256x256
            image = image.resize((256, 256), Image.Resampling.LANCZOS)

            # Convert to numpy array and normalize to [0,1]
            image = np.array(image, dtype=np.float32) / 255.0

            # Add noise
            if self.noise_type == 'gaussian':
                noisy = random_noise(image, mode='gaussian', var=self.noise_level**2, clip=True)
            elif self.noise_type == 'poisson':
                noisy = random_noise(image, mode='poisson', clip=True)
            else:
                noisy = image.copy()

            # Convert to tensors [1, H, W]
            image_tensor = torch.from_numpy(image).unsqueeze(0).float()
            noisy_tensor = torch.from_numpy(noisy.astype(np.float32)).unsqueeze(0).float()

            # Apply normalization to [-1, 1] if transform provided
            if self.transform:
                image_tensor = self.transform(image_tensor)
                noisy_tensor = self.transform(noisy_tensor)

            return {'clean': image_tensor, 'noisy': noisy_tensor}

        except Exception as e:
            print(f"Error processing image {idx} ({self.image_paths[idx]}): {e}")
            blank = torch.zeros(1, 256, 256)
            return {'clean': blank, 'noisy': blank}

# Preprocessing transform
transform = transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]

# Split dataset
train_size = 1500
val_size = min(500, len(all_image_paths) - train_size)

train_paths = all_image_paths[:train_size]
val_paths = all_image_paths[train_size:train_size + val_size]

print(f"\nCreating datasets...")
print(f"Train: {len(train_paths)} images")
print(f"Val: {len(val_paths)} images")

# Create datasets
train_dataset = XRayDataset(train_paths, transform=transform, noise_type='gaussian', noise_level=0.1)
val_dataset = XRayDataset(val_paths, transform=transform, noise_type='gaussian', noise_level=0.1)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

# Test loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_batch = next(iter(train_loader))
print(f"\n✓ DataLoaders created successfully!")
print(f"  Device: {device}")
print(f"  Batch shape: {test_batch['clean'].shape}")
print(f"  Data range: [{test_batch['clean'].min():.3f}, {test_batch['clean'].max():.3f}]")

Found 10192 images in Normal
Found 3616 images in COVID
Found 6012 images in Lung_Opacity

Total images found: 19820
Using 2000 images for this run

Creating datasets...
Train: 1500 images
Val: 500 images


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



✓ DataLoaders created successfully!
  Device: cpu
  Batch shape: torch.Size([16, 1, 256, 256])
  Data range: [-1.000, 1.000]


In [5]:
# ============================================================================
# CELL 2: MAE Encoder Architecture (Vision Transformer-based)
# ============================================================================

import torch.nn as nn
import torch.nn.functional as F

class PatchEmbedding(nn.Module):
    """Convert image into patches and embed them"""
    def __init__(self, img_size=256, patch_size=16, in_channels=1, embed_dim=256):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.grid_size = img_size // patch_size

        # Use conv2d for efficient patch embedding
        self.proj = nn.Conv2d(in_channels, embed_dim,
                             kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        """
        Args:
            x: [B, C, H, W]
        Returns:
            patches: [B, num_patches, embed_dim]
        """
        # x: [B, 1, 256, 256] -> [B, embed_dim, 16, 16]
        x = self.proj(x)
        # Flatten: [B, embed_dim, 16, 16] -> [B, embed_dim, 256]
        x = x.flatten(2)
        # Transpose: [B, embed_dim, 256] -> [B, 256, embed_dim]
        x = x.transpose(1, 2)
        return x


class MAEEncoder(nn.Module):
    """
    Masked Autoencoder Encoder
    Uses Vision Transformer (ViT) blocks to encode patches
    """
    def __init__(self, img_size=256, patch_size=16, in_channels=1,
                 embed_dim=256, depth=4, num_heads=8, mlp_ratio=4.0):
        super().__init__()

        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        self.num_patches = self.patch_embed.num_patches

        # Learnable positional embeddings
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches, embed_dim))

        # Transformer encoder blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio)
            for _ in range(depth)
        ])

        self.norm = nn.LayerNorm(embed_dim)

        # Initialize positional embeddings
        nn.init.trunc_normal_(self.pos_embed, std=0.02)

    def forward(self, x, mask_indices=None):
        """
        Args:
            x: [B, C, H, W] input images
            mask_indices: [B, num_masked] indices of patches to keep (for MAE pretraining)
        Returns:
            encoded: [B, num_patches (or num_visible), embed_dim]
        """
        # Patch embedding
        x = self.patch_embed(x)  # [B, num_patches, embed_dim]

        # Add positional embedding
        x = x + self.pos_embed

        # Apply mask if provided (for MAE pretraining)
        if mask_indices is not None:
            B, N, D = x.shape
            # Keep only visible patches
            x = torch.gather(x, 1, mask_indices.unsqueeze(-1).expand(-1, -1, D))

        # Apply transformer blocks
        for block in self.blocks:
            x = block(x)

        x = self.norm(x)
        return x


class TransformerBlock(nn.Module):
    """Standard Transformer block with multi-head attention and MLP"""
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0, dropout=0.1):
        super().__init__()

        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads,
                                         dropout=dropout, batch_first=True)

        self.norm2 = nn.LayerNorm(embed_dim)
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # Self-attention with residual
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        # MLP with residual
        x = x + self.mlp(self.norm2(x))
        return x


# Test the encoder
print("Testing MAE Encoder...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_encoder = MAEEncoder(
    img_size=256,
    patch_size=16,
    in_channels=1,
    embed_dim=256,
    depth=4,
    num_heads=8
).to(device)

test_input = torch.randn(2, 1, 256, 256).to(device)
test_output = test_encoder(test_input)

print(f"✓ Encoder created successfully!")
print(f"  Input shape:  {test_input.shape}")
print(f"  Output shape: {test_output.shape}")
print(f"  Parameters:   {sum(p.numel() for p in test_encoder.parameters()):,}")

del test_encoder, test_input, test_output
torch.cuda.empty_cache()

Testing MAE Encoder...
✓ Encoder created successfully!
  Input shape:  torch.Size([2, 1, 256, 256])
  Output shape: torch.Size([2, 256, 256])
  Parameters:   3,290,880


In [6]:
# ============================================================================
# CELL 3: MAE Decoder Architecture (Convolutional + Refinement)
# ============================================================================

class ConvDecoder(nn.Module):
    """
    Convolutional decoder for image reconstruction
    Converts encoded patches back to full resolution image
    """
    def __init__(self, img_size=256, patch_size=16, embed_dim=256, out_channels=1):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.grid_size = img_size // patch_size

        # Project from embed_dim to patch pixels
        self.decoder_pred = nn.Linear(embed_dim, patch_size * patch_size * out_channels)

        # Refinement CNN to improve reconstruction quality
        self.refine = nn.Sequential(
            nn.Conv2d(out_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, out_channels, kernel_size=3, padding=1),
        )

    def forward(self, x):
        """
        Args:
            x: [B, num_patches, embed_dim]
        Returns:
            reconstructed: [B, C, H, W]
        """
        B = x.shape[0]

        # Linear projection to patch pixels
        x = self.decoder_pred(x)  # [B, num_patches, patch_size^2 * C]

        # Reshape to patches
        # [B, num_patches, patch_size * patch_size * C]
        x = x.reshape(B, self.grid_size, self.grid_size,
                     self.patch_size, self.patch_size, -1)

        # Rearrange patches to image
        # [B, grid_h, grid_w, patch_h, patch_w, C] -> [B, C, H, W]
        x = x.permute(0, 5, 1, 3, 2, 4).contiguous()
        x = x.reshape(B, -1, self.img_size, self.img_size)

        # Refine with CNN
        x = self.refine(x)

        return x


# Test the decoder
print("Testing ConvDecoder...")
test_decoder = ConvDecoder(
    img_size=256,
    patch_size=16,
    embed_dim=256,
    out_channels=1
).to(device)

# Test with encoded features (256 patches, 256 dims each)
test_encoded = torch.randn(2, 256, 256).to(device)
test_reconstructed = test_decoder(test_encoded)

print(f"✓ Decoder created successfully!")
print(f"  Input shape:  {test_encoded.shape}")
print(f"  Output shape: {test_reconstructed.shape}")
print(f"  Parameters:   {sum(p.numel() for p in test_decoder.parameters()):,}")

del test_decoder, test_encoded, test_reconstructed
torch.cuda.empty_cache()

Testing ConvDecoder...
✓ Decoder created successfully!
  Input shape:  torch.Size([2, 256, 256])
  Output shape: torch.Size([2, 1, 256, 256])
  Parameters:   122,433
